In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)
#X=imp.fit_transform(X)
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

#Each training data set has NaN values removed (could also split by label and impute this)
LF_train_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/LF_train.csv',  sep=',',  header=0)
LH_train_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/LH_train.csv',  sep=',',  header=0)
RF_train_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/RF_train.csv',  sep=',',  header=0)
RH_train_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/RH_train.csv',  sep=',',  header=0)
LF_train_pred=LF_train_df[LF_train_df.columns[6]].to_numpy()
LH_train_pred=LH_train_df[LH_train_df.columns[6]].to_numpy()
RF_train_pred=RF_train_df[RF_train_df.columns[6]].to_numpy()
RH_train_pred=RH_train_df[RH_train_df.columns[6]].to_numpy()
LF_train_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
LH_train_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
RF_train_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
RH_train_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)

#Test points
LF_test_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/LF_test.csv',  sep=',',  header=0)
LF_ids = LF_test_df['id']
LH_test_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/LH_test.csv',  sep=',',  header=0)
LH_ids = LH_test_df['id']
RF_test_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/RF_test.csv',  sep=',',  header=0)
RF_ids = RF_test_df['id']
RH_test_df = pd.read_csv('../input/cs4780-spring-2023-kaggle-competition/RH_test.csv',  sep=',',  header=0)
RH_ids = RH_test_df['id']


for col in LF_train_df.columns:
    if(col!=('LF'or 'LH' or 'RH' or 'RF')):
        LF_train_df[col]=pd.to_numeric(LF_train_df[col], errors='coerce')
        LH_train_df[col]=pd.to_numeric(LH_train_df[col], errors='coerce')
        RF_train_df[col]=pd.to_numeric(RF_train_df[col], errors='coerce')
        RH_train_df[col]=pd.to_numeric(RH_train_df[col], errors='coerce')
        LF_train_df[col] = LF_train_df[col].fillna(LF_train_df.groupby('LF')[col].transform('mean'))
        LH_train_df[col] = LH_train_df[col].fillna(LH_train_df.groupby('LH')[col].transform('mean'))
        RF_train_df[col] = RF_train_df[col].fillna(RF_train_df.groupby('RF')[col].transform('mean'))
        RH_train_df[col] =RH_train_df[col].fillna(RH_train_df.groupby('RH')[col].transform('mean'))
        LF_test_df[col] = pd.to_numeric(LF_test_df[col], errors='coerce')
        LH_test_df[col] = pd.to_numeric(LH_test_df[col], errors='coerce')
        RF_test_df[col] = pd.to_numeric(RF_test_df[col], errors='coerce')
        RH_test_df[col] = pd.to_numeric(RH_test_df[col], errors='coerce')

LF_train_df.drop(columns=['LF'], inplace=True)
LH_train_df.drop(columns=['LH'], inplace=True)
RF_train_df.drop(columns=['RF'], inplace=True)
RH_train_df.drop(columns=['RH'], inplace=True)


#Each test data set has missing values replaced with the mean of the column
LF_test_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
LF_test_df.fillna(LF_test_df.median(), inplace=True)

LH_test_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
LH_test_df.fillna(LH_test_df.median(), inplace=True)

RF_test_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
RF_test_df.fillna(RF_test_df.median(), inplace=True)

RH_test_df.drop(columns=['gait','Gait','dob','id','forceplate_date'], inplace=True)
RH_test_df.fillna(RH_test_df.median(), inplace=True)

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

sc = StandardScaler()
sc.fit(LF_train_df)
LF_train_std = sc.transform(LF_train_df)
LF_test_std = sc.transform(LF_test_df)

# Training a SVM classifier using SVC class
svm = SVC(kernel= 'linear', random_state=2, C=0.0076)
svm.fit(LF_train_std, LF_train_pred)
LF_pred = svm.predict(LF_test_std)

sc = StandardScaler()
sc.fit(LH_train_df)
LH_train_std = sc.transform(LH_train_df)
LH_test_std = sc.transform(LH_test_df)
# Training a SVM classifier using SVC class
svm = SVC(kernel= 'linear', random_state=2, C=0.0076)
svm.fit(LH_train_std, LH_train_pred)
LH_pred = svm.predict(LH_test_std)

sc = StandardScaler()
sc.fit(RF_train_df)
RF_train_std = sc.transform(RF_train_df)
RF_test_std = sc.transform(RF_test_df)
# Training a SVM classifier using SVC class
svm = SVC(kernel= 'linear', random_state=2, C=0.0076)
svm.fit(RF_train_std, RF_train_pred)
RF_pred = svm.predict(RF_test_std)

sc = StandardScaler()
sc.fit(RH_train_df)
RH_train_std = sc.transform(RH_train_df)
RH_test_std = sc.transform(RH_test_df)
# Training a SVM classifier using SVC class
svm = SVC(kernel= 'linear', random_state=2, C=0.0076)
svm.fit(RH_train_std, RH_train_pred)
RH_pred = svm.predict(RH_test_std)

In [ ]:
import csv 
LF_data = []

ids = LF_ids
for i in range(len(LF_test_df)):
    pt = [ids[i],LF_pred[i]]
    LF_data += [pt]

filename = "LF_test_labels.csv"
fields =['id','LF']
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
        
    csvwriter.writerow(fields) 
        
    csvwriter.writerows(LF_data)    
###

LH_data = []

ids = LH_ids
for i in range(len(LH_test_df)):
    pt = [ids[i],LH_pred[i]]
    LH_data += [pt]

filename = "LH_test_labels.csv"
fields =['id','LH']
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
        
    csvwriter.writerow(fields) 
        
    csvwriter.writerows(LH_data)    
###

RF_data = []

ids = RF_ids
for i in range(len(RF_test_df)):
    pt = [ids[i],RF_pred[i]]
    RF_data += [pt]

filename = "RF_test_labels.csv"
fields =['id','RF']
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
        
    csvwriter.writerow(fields) 
        
    csvwriter.writerows(RF_data)  

###
RH_data = []
ids = RH_ids
for i in range(len(RH_test_df)):
    pt = [ids[i],RH_pred[i]]
    RH_data += [pt]

filename = "RH_test_labels.csv"
fields =['id','RH']
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
        
    csvwriter.writerow(fields) 
        
    csvwriter.writerows(RH_data)

In [ ]:
import pandas as pd

"""
This is a file that will help you convert your individual predictions to the final prediction. 
In the same directory as this file, you should have the following 4 files:
  - LF_test_labels.csv - with at least two columns, 'id' and 'LF'
  - LH_test_labels.csv - with at least two columns, 'id' and 'LH'
  - RF_test_labels.csv - with at least two columns, 'id' and 'RF'
  - RH_test_labels.csv - with at least two columns, 'id' and 'RH'

Running this script will convert these four files into a single CSV file, submission.csv, by
mutating the IDs so that they also include the leg that is being checked.
"""

legs = ["LF", "LH", "RF", "RH"]

dfs = []

for leg in legs:
    # read in the file
    test_prediction = pd.read_csv(f"{leg}_test_labels.csv")
    # append the abbreviation for the leg
    test_prediction['id'] = test_prediction['id'].astype(str) + f"_{leg}"
    # rename the label column
    test_prediction['label'] = test_prediction[leg]
    # exclude any potential additional columns
    dfs.append(test_prediction[['id', 'label']])

final_df = pd.concat(dfs)
final_df.to_csv("submission.csv", index=False)